## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps
import sys
sys.path.append("..")

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# Import the database csv file. 
city_data_df = pd.read_csv("../weather_database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Esperance,AU,-33.8667,121.9000,48.43,74,0,13.87,clear sky
1,1,Fez,MA,34.0372,-4.9998,80.85,50,40,9.22,light thunderstorm
2,2,Walvis Bay,NaN,-22.9575,14.5053,58.15,77,0,0.00,clear sky
3,3,Hilo,US,19.7297,-155.0900,86.29,93,100,12.66,moderate rain
4,4,Tshikapa,CD,-6.4167,20.8000,75.40,36,2,2.26,clear sky


In [3]:
# Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Fez,MA,34.0372,-4.9998,80.85,50,40,9.22,light thunderstorm
3,3,Hilo,US,19.7297,-155.0900,86.29,93,100,12.66,moderate rain
4,4,Tshikapa,CD,-6.4167,20.8000,75.40,36,2,2.26,clear sky
8,8,Estreito,BR,-5.7833,-43.2500,84.92,46,15,1.83,few clouds
10,10,Zabol,AF,32.2500,67.2500,80.67,50,0,5.91,clear sky
11,11,Kapaa,US,22.0752,-159.3190,82.72,71,100,14.97,overcast clouds
13,13,Albany,US,42.6001,-73.9662,82.45,41,0,1.01,clear sky
16,16,Sakakah,SA,29.9697,40.2064,89.17,17,0,14.97,clear sky
18,18,Rikitea,PF,-23.1203,-134.9692,75.06,73,0,16.71,clear sky
24,24,Wattegama,LK,6.7989,81.4808,75.60,93,100,2.86,overcast clouds


In [5]:
# Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                262
City                   262
Country                262
Lat                    262
Lon                    262
Max Temp               262
Humidity               262
Cloudiness             262
Wind Speed             262
Current Description    262
dtype: int64

In [6]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(how='any')

In [7]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lon"]].copy()

# Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lon,Hotel Name
1,Fez,MA,80.85,light thunderstorm,34.0372,-4.9998,
3,Hilo,US,86.29,moderate rain,19.7297,-155.0900,
4,Tshikapa,CD,75.40,clear sky,-6.4167,20.8000,
8,Estreito,BR,84.92,few clouds,-5.7833,-43.2500,
10,Zabol,AF,80.67,clear sky,32.2500,67.2500,
11,Kapaa,US,82.72,overcast clouds,22.0752,-159.3190,
13,Albany,US,82.45,clear sky,42.6001,-73.9662,
16,Sakakah,SA,89.17,clear sky,29.9697,40.2064,
18,Rikitea,PF,75.06,clear sky,-23.1203,-134.9692,
24,Wattegama,LK,75.60,overcast clouds,6.7989,81.4808,


In [8]:
# Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lon"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('',np.nan,inplace=True)
clean_hotel_df = hotel_df.dropna(subset=['Hotel Name'],how='any')

In [11]:
# Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lon"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))